<a href="https://colab.research.google.com/github/Cactice/bert-ja-experiment/blob/master/first_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install keras-bert
!pip install gdown
!pip install sentencepiece


  Created wheel for keras-bert: filename=keras_bert-0.85.0-cp36-none-any.whl size=34303 sha256=0fdf529ebcea06372c3dca1efcbe31f1870963b6fb94296ae544b46d1ce87ca2
  Stored in directory: /root/.cache/pip/wheels/1f/7a/13/40e59e2587ebfdba717d5ecc4c89c614f06c8137e515fa8b26
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12942 sha256=fa699367c5a921649a61b444e7ba80aab3bccc623124d700b4987a16b4b4d920
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=602cebd51b95087eb8f0a71ea276a83bfacff88a8072c533380a76757f832ec5
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15611 sha256=599b51abdba62bce3e196ff882cf57d9b591a4b20fed6d8e5bd1c4accd68050f
  Stor

In [2]:
  # https://drive.google.com/file/d/11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In/view?usp=sharing
  # https://drive.google.com/file/d/1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS/view?usp=sharing
  # https://drive.google.com/file/d/1F4b_u-5zzqabA6OfLxDkLh0lzqVIEZuN/view?usp=sharing
  # https://drive.google.com/file/d/1LB00MDQJjb-xLmgBMhdQE3wKDOLjgum-/view?usp=sharing
  # https://drive.google.com/file/d/1V9TIUn5wc-mB_wabYiz9ikvLsscONOKB/view?usp=sharing
  # https://drive.google.com/file/d/1jjZmgSo8C9xMIos8cUMhqJfNbyyqR0MY/view?usp=sharing
  # https://drive.google.com/file/d/1uzPpW38LcS4YS431GgdG0Hsj4gNgE5X1/view?usp=sharing

# !git clone --recurse-submodules https://github.com/yoheikikuta/bert-japanese
# !git clone https://github.com/Cactice/bert-ja-experiment

# !gdown https://drive.google.com/uc?id=11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In
# !gdown https://drive.google.com/uc?id=1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS
# !gdown https://drive.google.com/uc?id=1F4b_u-5zzqabA6OfLxDkLh0lzqVIEZuN
# !gdown https://drive.google.com/uc?id=1LB00MDQJjb-xLmgBMhdQE3wKDOLjgum-
# !gdown https://drive.google.com/uc?id=1V9TIUn5wc-mB_wabYiz9ikvLsscONOKB
# !gdown https://drive.google.com/uc?id=1jjZmgSo8C9xMIos8cUMhqJfNbyyqR0MY
# !gdown https://drive.google.com/uc?id=1uzPpW38LcS4YS431GgdG0Hsj4gNgE5X1

In [3]:
%cd /content
!ls 

/content
sample_data


In [4]:
from google.colab import drive
drive.mount('/drive')

#Execute this to save to drive

#! cp -r /content /drive/My\ Drive/bert-ja

#Execute this to load from drive

!cp -r /drive/My\ Drive/bert-ja/content /
# use data_processor.ipynb to generate this file below
!cp -r /drive/My\ Drive/KNBC/out/* /KNBC/

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [5]:
import sys
sys.path.append('modules')
from keras_bert import load_trained_model_from_checkpoint

config_path = 'bert-ja-experiment/bert_config.json'
# `model.ckpt-1400000` のように拡張子を付けないのがポイントです。
checkpoint_path = 'model.ckpt-1400000'

bert = load_trained_model_from_checkpoint(config_path, checkpoint_path)
bert.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 512)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [6]:
import pandas as pd
import sentencepiece as spm
#from keras import utils
import logging
import numpy as np
from tensorflow.python.keras import utils

maxlen = 512
bert_dim = 768

sp = spm.SentencePieceProcessor()
sp.Load('wiki-ja.model')

def _get_vector(feature):
    common_seg_input = np.zeros((1, maxlen), dtype = np.float32)
    indices = np.zeros((1, maxlen), dtype = np.float32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        try:
            indices[0, t] = sp.piece_to_id(token)
        except:
            logging.warn(f'{token} is unknown.')
            indices[0, t] = sp.piece_to_id('<unk>')
    vector =  bert.predict([indices, common_seg_input])[0]

    return vector

def _load_labeldata(train_dir, test_dir):
    train_features_df = pd.read_csv(f'{train_dir}/features.csv')
    train_labels_df = pd.read_csv(f'{train_dir}/labels.csv')
    test_features_df = pd.read_csv(f'{test_dir}/features.csv')
    test_labels_df = pd.read_csv(f'{test_dir}/labels.csv')
    label2index = {k: i for i, k in enumerate(train_labels_df['label'].unique())}
    index2label = {i: k for i, k in enumerate(train_labels_df['label'].unique())}
    class_count = len(label2index)
    train_labels = utils.np_utils.to_categorical([label2index[label] for label in train_labels_df['label']], num_classes=class_count)
    test_label_indices = [label2index[label] for label in test_labels_df['label']]
    test_labels = utils.np_utils.to_categorical(test_label_indices, num_classes=class_count)

    train_features = []
    test_features = []

    for feature in train_features_df['feature']:
        train_features.append(_get_vector(feature))
    for feature in test_features_df['feature']:
        test_features.append(_get_vector(feature))

    print(f'Trainデータ数: {len(train_features_df)}, Testデータ数: {len(test_features_df)}, ラベル数: {class_count}')

    return {
        'class_count': class_count,
        'label2index': label2index,
        'index2label': index2label,
        'train_labels': train_labels,
        'test_labels': test_labels,
        'test_label_indices': test_label_indices,
        'train_features': np.array(train_features),
        'test_features': np.array(test_features),
        'input_len': maxlen
    }

In [7]:
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras import Input, Model

def _create_model(train_features):
    class_count = 4

    input_tensor = Input(train_features[0].shape)
    x1 = Bidirectional(LSTM(356))(input_tensor)
    output_tensor = Dense(class_count, activation='softmax')(x1)

    model = Model(input_tensor, output_tensor)
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['mae', 'mse', 'acc'])

    return model

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard


trains_dir = '/KNBC/train'
tests_dir = '/KNBC/test'

data = _load_labeldata(trains_dir, tests_dir)
model_filename = 'models/knbc-check-bert_v3.model'
model = _create_model(data['train_features'].shape, data['class_count'])

model.summary()


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

model_filename = 'knbc-check-bert_v2.model'

model = _create_model(data['train_features'])
model.summary()
history = model.fit(data['train_features'],
          data['train_labels'],
          epochs = 100,
          batch_size = 128,
          validation_data=(data['test_features'], data['test_labels']),
          shuffle=False,
          verbose = 1,
          callbacks = [
              EarlyStopping(patience=5, monitor='val_acc', mode='max'),
              ModelCheckpoint(monitor='val_acc', mode='max', filepath=model_filename, save_best_only=True)
          ])

In [ ]:
# trains_dir = '../word-or-character/data/trains'
# tests_dir = '../word-or-character/data/tests'

# data = _load_labeldata(trains_dir, tests_dir)

!curl -s | tar xvz -C /tmp 

In [ ]:
#wget -qO-  http://nlp.ist.i.kyoto-u.ac.jp/kuntt/KNBC_v1.0_090925_utf8.tar.bz2 | tar xvz - -C /content/data
!wget http://nlp.ist.i.kyoto-u.ac.jp/kuntt/KNBC_v1.0_090925_utf8.tar.bz2
!tar -xf /content/KNBC_v1.0_090925_utf8.tar.bz2